In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

## Levin t

In [6]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch", method="Levin:t")
loss_fn = nn.NLLLoss()

In [8]:
log_file = open("SGD-Levin:t.txt", "w")

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.9848
Validation accuracy: 0.7210, validation loss: 1.7053
Epoch 2
Training loss: 1.5156
Validation accuracy: 0.7837, validation loss: 1.3520
Epoch 3
Training loss: 1.2413
Validation accuracy: 0.8080, validation loss: 1.1394
Epoch 4
Training loss: 1.0708
Validation accuracy: 0.8207, validation loss: 1.0015
Epoch 5
Training loss: 0.9566
Validation accuracy: 0.8277, validation loss: 0.9059
Epoch 6
Training loss: 0.8752
Validation accuracy: 0.8352, validation loss: 0.8357
Epoch 7
Training loss: 0.8141
Validation accuracy: 0.8414, validation loss: 0.7820
Epoch 8
Training loss: 0.7665
Validation accuracy: 0.8459, validation loss: 0.7395
Epoch 9
Training loss: 0.7283
Validation accuracy: 0.8499, validation loss: 0.7047
Epoch 10
Training loss: 0.6968
Validation accuracy: 0.8532, validation loss: 0.6759
Epoch 11
Training loss: 0.6704
Validation accuracy: 0.8570, validation loss: 0.6514
Epoch 12
Training loss: 0.6478
Validation accuracy: 0.8586, validation loss: 0.6304
E

In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8786, device='cuda:0'), tensor(0.4768, device='cuda:0'))
Valid: (tensor(0.8828, device='cuda:0'), tensor(0.4696, device='cuda:0'))


In [11]:
optimizer.accelerate()

In [12]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [13]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8620, device='cuda:0'), tensor(0.5093, device='cuda:0'))
Valid: (tensor(0.8686, device='cuda:0'), tensor(0.5014, device='cuda:0'))


## Levin u

In [14]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [15]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch", method="Levin:u")
loss_fn = nn.NLLLoss()

In [16]:
log_file = open("SGD-Levin:u.txt", "w")

In [17]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.9598
Validation accuracy: 0.7303, validation loss: 1.6877
Epoch 2
Training loss: 1.4994
Validation accuracy: 0.7810, validation loss: 1.3406
Epoch 3
Training loss: 1.2305
Validation accuracy: 0.8053, validation loss: 1.1318
Epoch 4
Training loss: 1.0632
Validation accuracy: 0.8179, validation loss: 0.9961
Epoch 5
Training loss: 0.9510
Validation accuracy: 0.8287, validation loss: 0.9017
Epoch 6
Training loss: 0.8708
Validation accuracy: 0.8355, validation loss: 0.8325
Epoch 7
Training loss: 0.8106
Validation accuracy: 0.8420, validation loss: 0.7793
Epoch 8
Training loss: 0.7636
Validation accuracy: 0.8466, validation loss: 0.7371
Epoch 9
Training loss: 0.7258
Validation accuracy: 0.8510, validation loss: 0.7028
Epoch 10
Training loss: 0.6946
Validation accuracy: 0.8550, validation loss: 0.6741
Epoch 11
Training loss: 0.6684
Validation accuracy: 0.8572, validation loss: 0.6498
Epoch 12
Training loss: 0.6461
Validation accuracy: 0.8596, validation loss: 0.6290
E

In [18]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8792, device='cuda:0'), tensor(0.4764, device='cuda:0'))
Valid: (tensor(0.8831, device='cuda:0'), tensor(0.4691, device='cuda:0'))


In [19]:
optimizer.accelerate()

In [20]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [21]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8800, device='cuda:0'), tensor(0.4736, device='cuda:0'))
Valid: (tensor(0.8837, device='cuda:0'), tensor(0.4663, device='cuda:0'))


## Levin v

In [22]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [23]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch", method="Levin:v")
loss_fn = nn.NLLLoss()

In [24]:
log_file = open("SGD-Levin:v.txt", "w")

In [25]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.9812
Validation accuracy: 0.7346, validation loss: 1.7001
Epoch 2
Training loss: 1.5075
Validation accuracy: 0.7923, validation loss: 1.3468
Epoch 3
Training loss: 1.2341
Validation accuracy: 0.8110, validation loss: 1.1350
Epoch 4
Training loss: 1.0648
Validation accuracy: 0.8241, validation loss: 0.9979
Epoch 5
Training loss: 0.9516
Validation accuracy: 0.8322, validation loss: 0.9030
Epoch 6
Training loss: 0.8710
Validation accuracy: 0.8390, validation loss: 0.8334
Epoch 7
Training loss: 0.8105
Validation accuracy: 0.8448, validation loss: 0.7801
Epoch 8
Training loss: 0.7634
Validation accuracy: 0.8487, validation loss: 0.7379
Epoch 9
Training loss: 0.7256
Validation accuracy: 0.8519, validation loss: 0.7035
Epoch 10
Training loss: 0.6944
Validation accuracy: 0.8553, validation loss: 0.6748
Epoch 11
Training loss: 0.6683
Validation accuracy: 0.8579, validation loss: 0.6507
Epoch 12
Training loss: 0.6460
Validation accuracy: 0.8612, validation loss: 0.6298
E

In [26]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8787, device='cuda:0'), tensor(0.4765, device='cuda:0'))
Valid: (tensor(0.8837, device='cuda:0'), tensor(0.4700, device='cuda:0'))


In [27]:
optimizer.accelerate()

In [28]:
optimizer.store_parameters()
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [29]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.8757, device='cuda:0'), tensor(0.4929, device='cuda:0'))
Valid: (tensor(0.8819, device='cuda:0'), tensor(0.4865, device='cuda:0'))
